# Assignment 4: Pipelines and Text Data (60 total marks)
### Due: March 21 at 11:59pm

### Name: Maryam Alsayed

In [117]:
import numpy as np
import pandas as pd

In [118]:
import warnings
warnings.filterwarnings('ignore') #ignoring some deprication warnings

## Part 1: Pipelines (26 marks)

The purpose of this part of the assignment is to practice following the grid-search workflow: 
- Split data into training and test set
- Use the training portion to find the best model using grid search and cross-validation
- Retrain the best model
- Evaluate the retrained model on the test set

### 1.1: Load data (4 marks)
For this task, we will be using stock data from the Dow Jones Index. This dataset uses information about different stocks to try to predict what the percent change in price will be from week to week.

More information on the dataset can be found here: https://archive.ics.uci.edu/dataset/312/dow+jones+index

In [119]:
# TO DO: Load the dataset into a dataframe called stock_data (0.5 marks)
stock_data = pd.read_csv('dow_jones_index.data')
# TO DO: Inspect the first few columns (0.5 marks)
stock_data.head()

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1/7/2011,$15.82,$16.72,$15.78,$16.42,239655616,3.79267,NaN,NaN,$16.71,$15.97,-4.428490,26,0.182704
1,1,AA,1/14/2011,$16.71,$16.71,$15.64,$15.97,242963398,-4.42849,1.380223,239655616.0,$16.19,$15.79,-2.470660,19,0.187852
2,1,AA,1/21/2011,$16.19,$16.38,$15.60,$15.79,138428495,-2.47066,-43.024959,242963398.0,$15.87,$16.13,1.638310,12,0.189994
3,1,AA,1/28/2011,$15.87,$16.63,$15.82,$16.13,151379173,1.63831,9.355500,138428495.0,$16.18,$17.14,5.933250,5,0.185989
4,1,AA,2/4/2011,$16.18,$17.39,$16.18,$17.14,154387761,5.93325,1.987452,151379173.0,$17.33,$17.37,0.230814,97,0.175029


In [120]:
# TO DO: Check the data types of each column and if there are missing values (0.5 marks)
print("Checking data types:\n")
stock_data.info()
print("\nChecking number of missing values:")
stock_data.isnull().sum()

Checking data types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   quarter                             750 non-null    int64  
 1   stock                               750 non-null    object 
 2   date                                750 non-null    object 
 3   open                                750 non-null    object 
 4   high                                750 non-null    object 
 5   low                                 750 non-null    object 
 6   close                               750 non-null    object 
 7   volume                              750 non-null    int64  
 8   percent_change_price                750 non-null    float64
 9   percent_change_volume_over_last_wk  720 non-null    float64
 10  previous_weeks_volume               720 non-null    float64
 11  next_weeks_open        

quarter                                0
stock                                  0
date                                   0
open                                   0
high                                   0
low                                    0
close                                  0
volume                                 0
percent_change_price                   0
percent_change_volume_over_last_wk    30
previous_weeks_volume                 30
next_weeks_open                        0
next_weeks_close                       0
percent_change_next_weeks_price        0
days_to_next_dividend                  0
percent_return_next_dividend           0
dtype: int64

You should notice in this dataset that there are multiple columns that look numerical, but include a `$` that turns the value into a string (type object). You can use the code below to convert these columns into numerical ones:

In [121]:
# TO DO: Fill-in which columns need the $ to be removed (1 mark)
columns = ['open', 'high', 'close', 'low', 'next_weeks_open', 'next_weeks_close']

# Code to remove $ - DO NOT CHANGE
stock_data[columns] = stock_data[columns].replace('[\$]', '', regex=True).astype(float)

# TO DO: Inspect first few rows to make sure it worked (0.5 marks)
stock_data.head()

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1/7/2011,15.82,16.72,15.78,16.42,239655616,3.79267,NaN,NaN,16.71,15.97,-4.428490,26,0.182704
1,1,AA,1/14/2011,16.71,16.71,15.64,15.97,242963398,-4.42849,1.380223,239655616.0,16.19,15.79,-2.470660,19,0.187852
2,1,AA,1/21/2011,16.19,16.38,15.60,15.79,138428495,-2.47066,-43.024959,242963398.0,15.87,16.13,1.638310,12,0.189994
3,1,AA,1/28/2011,15.87,16.63,15.82,16.13,151379173,1.63831,9.355500,138428495.0,16.18,17.14,5.933250,5,0.185989
4,1,AA,2/4/2011,16.18,17.39,16.18,17.14,154387761,5.93325,1.987452,151379173.0,17.33,17.37,0.230814,97,0.175029


In [122]:
# TO DO: Check data type of each column to make sure that the type of the columns selected has changed (0.5 marks)
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   quarter                             750 non-null    int64  
 1   stock                               750 non-null    object 
 2   date                                750 non-null    object 
 3   open                                750 non-null    float64
 4   high                                750 non-null    float64
 5   low                                 750 non-null    float64
 6   close                               750 non-null    float64
 7   volume                              750 non-null    int64  
 8   percent_change_price                750 non-null    float64
 9   percent_change_volume_over_last_wk  720 non-null    float64
 10  previous_weeks_volume               720 non-null    float64
 11  next_weeks_open                     750 non-n

The first thing we need to do is deal with missing values. Looking at the dataset, there are two columns with 30 missing values. For this case, we will drop these rows instead of filling them in.

In [123]:
# TO DO: Drop rows with missing data (0.5 marks)
stock_data.dropna()

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
1,1,AA,1/14/2011,16.71,16.71,15.64,15.97,242963398,-4.428490,1.380223,239655616.0,16.19,15.79,-2.470660,19,0.187852
2,1,AA,1/21/2011,16.19,16.38,15.60,15.79,138428495,-2.470660,-43.024959,242963398.0,15.87,16.13,1.638310,12,0.189994
3,1,AA,1/28/2011,15.87,16.63,15.82,16.13,151379173,1.638310,9.355500,138428495.0,16.18,17.14,5.933250,5,0.185989
4,1,AA,2/4/2011,16.18,17.39,16.18,17.14,154387761,5.933250,1.987452,151379173.0,17.33,17.37,0.230814,97,0.175029
5,1,AA,2/11/2011,17.33,17.48,16.97,17.37,114691279,0.230814,-25.712195,154387761.0,17.39,17.28,-0.632547,90,0.172712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,2,XOM,5/27/2011,80.22,82.63,80.07,82.63,68230855,3.004240,-21.355713,86758820.0,83.28,81.18,-2.521610,75,0.568801
746,2,XOM,6/3/2011,83.28,83.75,80.18,81.18,78616295,-2.521610,15.221032,68230855.0,80.93,79.78,-1.420980,68,0.578960
747,2,XOM,6/10/2011,80.93,81.87,79.72,79.78,92380844,-1.420980,17.508519,78616295.0,80.00,79.02,-1.225000,61,0.589120
748,2,XOM,6/17/2011,80.00,80.82,78.33,79.02,100521400,-1.225000,8.811952,92380844.0,78.65,76.78,-2.377620,54,0.594786


### 1.2: Pre-processing (4 marks)

In this dataset, we have columns with:
- Categorical values
- Numerical values

We need to create a column transformer that will use the proper preprocessing methods on each type of column.

In [124]:
# TO DO: Create Column Transformer using an encoder and StandardScaler (1 mark)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

categorical_data = ['stock', 'date']
numerical_data = ['quarter', 'open', 'high', 'low', 'close', 'volume', 'percent_change_price', 'percent_change_volume_over_last_wk', 'previous_weeks_volume', 'next_weeks_open', 'next_weeks_close', 'percent_change_next_weeks_price', 'days_to_next_dividend', 'percent_return_next_dividend']

preprocessor = ColumnTransformer(
    transformers = [
        ('categorical', OneHotEncoder(), categorical_data),
        ('numerical', StandardScaler(), numerical_data),
    ])

In [125]:
# TO DO: Initialize your pipeline with your column transformer and the Ridge Regression model (1 mark)
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

pipeline = Pipeline(steps = [('preprocessor', preprocessor),
                            ('model', Ridge())])

In [126]:
# TO DO: Separate data into feature matrix and target vector (1 mark)
X = stock_data.drop('percent_change_next_weeks_price', axis = 1)
y = stock_data['percent_change_next_weeks_price']

In [127]:
# TO DO: Split data into training and testing sets (use random_state=0 and 10% of the data for testing) (0.5 marks)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

Create another column transformer that does not implement scaling

In [128]:
# TO DO: Create a new column transformer that only performs encoding (0.5 marks)
preprocessor_encoding = ColumnTransformer(
    transformers = [
        ('categorical', OneHotEncoder(), categorical_data)
    ])

### 1.3: Grid Search (4 marks)

For the grid search, we want to compare the performance of the Random Forest model to a Ridge Regression model with the two different column transformers. Think about if we need to use scaling for both models. Select parameter values to test that make sense for both models.

In [129]:
# TO DO: Create parameter grid and initialize grid object (3 marks)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'model': [Ridge()],
    'model__alpha': [0.1, 1, 10],
    'preprocessor': [preprocessor]},
    {'model': [Ridge()],
    'model__alpha': [0.1, 1, 10],
    'preprocessor': [preprocessor_encoding]},
    {'model': [RandomForestRegressor()],
    'model__n_estimators': [100, 200],
    'preprocessor': [preprocessor_encoding]}
]

grid_search = GridSearchCV(pipeline, 
                           param_grid, 
                           cv = 5, 
                           scoring = 'r2')

In [130]:
# TO DO: Fit grid object to training data (1 mark)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         OneHotEncoder(),
                                                                         ['stock',
                                                                          'date']),
                                                                        ('numerical',
                                                                         StandardScaler(),
                                                                         ['quarter',
                                                                          'open',
                                                                          'high',
                                                                          'low',
                                                                         

### 1.4: Visualize Results (2 marks)

The final step is to print out the results from the grid search. You will need to print out the following items:
- Best parameters
- Best cross-validation train score 
- Best cross-validation test score
- Test set accuracy

In [131]:
# TO DO: Print the results from the grid search (2 marks)
# Print the results from the grid search
print("Best Parameters:", grid_search.best_params_)
print("Best cross-validation train score:", grid_search.best_score_)
print("Best cross-validation test score:", grid_search.cv_results_['mean_test_score'][grid_search.best_index_])
print("Test set accuracy:", grid_search.score(X_test, y_test))

Best Parameters: {'model': Ridge(alpha=1), 'model__alpha': 1, 'preprocessor': ColumnTransformer(transformers=[('categorical', OneHotEncoder(),
                                 ['stock', 'date'])])}
Best cross-validation train score: 0.30351059743156616
Best cross-validation test score: 0.30351059743156616
Test set accuracy: 0.3453887088419628


### Questions (8 marks)

1. Which models did you use scaling for? Why?
1. Which model and what parameters produced the best results?
1. Was this model a good fit? Why or why not?
1. Is there anything else we could do to try to improve model performance? Provide two ideas.

1. I used scaling for the Ridge Regression model. This is because it is sensitive to scale of the input features. Scaling made it so that all of the features had eual contribution to model performance.
2. The RandomForestRegressor model with n_estimators = 200 produced the best results.
3. This model was not a very good fit. This is evident when analyzing the test accuracy and cross-validation train/test scores. The best cross-validation scores were approximately 0.3, and the test accuracy was just under 0.3. This signals that the model is underfitting the data.
4. Two ways to improve model performance are feature engineering and hyperparameter tuning.

### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

1. I sourced my code from class content, such as labs and lecture examples, as well as through the internet and generative AI.
2. I completed the steps in the order they were given.
3. I used generative AI when I hit roadblocks or errors. I provided prompts such as "given the following information, can you analyze and explain the results" as well as providing it with pieces of completed work and asking it to finish the TODO at points where I was stuck. I did not have to modify the code much as it was often used for debugging or explanations, and when it was used for actual code generation the lines were small enough that adjustment was not required.
4. Having a good understanding of the lecture content and going over the lecture/lab examples was helpful in eliminating challenges through the process of completing part 1.

## Part 2: Text Data (32 marks)

The purpose of this part of the assignment is to practice working with text data.

### 2.1: Load data (1 mark)
For this task, we will be using the hobbies dataset from the yellowbrick library. More information on the dataset can be found here: https://www.scikit-yb.org/en/latest/api/datasets/hobbies.html

In [132]:
# TO DO: Load the dataset (1 mark)
import yellowbrick
from yellowbrick.datasets import load_hobbies
data = load_hobbies()

### 2.2 Pre-processing (3 marks)

We will need to transform the data from strings to numeric. First, we will transform the data using `CountVectorizer(min_df=5)`.

In [133]:
# TO DO: Create CountVectorizer object (0.5 marks)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df = 5)

In [134]:
# TO DO: Fit vectorizer to data (0.5 marks)
fit = vectorizer.fit(data.data)

In [135]:
# TO DO: What is the length of the vocabulary? (0.5 marks)
vocab_length = len(vectorizer.vocabulary_)
print("length:", vocab_length)

length: 3940


In [136]:
# TO DO: Transform the data (0.5 marks)
X = vectorizer.transform(data.data)

In [137]:
# TO DO: What is the shape of the transformed data? (0.5 marks)
print("Shape:", X.shape)

Shape: (448, 3940)


In [138]:
# TO DO: Split data into training and testing sets (use random_state=0 and 10% of the data for testing) (0.5 marks)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data.target, test_size = 0.1, random_state = 0)

### 2.3: Grid Search (5 marks)

For the grid search, we want to compare the performance of Logistic Regression for different values of C. Initialize the parameter grid with parameter values that make sense for this model.

In [139]:
# TO DO: Create parameter grid and initialize grid object (2 marks)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100]
}

logistic = LogisticRegression(max_iter = 1000)
grid_search = GridSearchCV(logistic, param_grid, cv=5, scoring = 'accuracy')

In [140]:
# TO DO: Fit grid object to training data (1 mark)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [0.1, 1, 10, 100]}, scoring='accuracy')

In [141]:
# TO DO: Print the results from the grid search (2 marks)
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)
print("Test Set Accuracy:", grid_search.score(X_test, y_test))

Best Parameters: {'C': 1}
Best Cross-Validation Accuracy: 0.7916049382716049
Test Set Accuracy: 0.9111111111111111


### 2.4: Additional Model Comparisons (9 marks)

### 2.4.1: Naive Bayes (3 marks)
We would like to compare the performance of Logistic Regression with one of the Naive Bayes models. Pick the Naive Bayes model that you think would best suit text data and implement below. Since we are not adjusting hyperparameters, we can use `cross_validate`.

In [142]:
# TO DO: Implement Naive Bayes model with cross-validate (2 marks)
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate

multinomial = MultinomialNB()
results = cross_validate(multinomial, X_train, y_train, cv = 5, scoring = 'accuracy', return_train_score = True)

# TO DO: Print training and validation accuracies
print("Training accuracy:", results['train_score'].mean())
print("Validation accuracy:", results['test_score'].mean())

Training accuracy: 0.960915716400977
Validation accuracy: 0.8808641975308642


In [143]:
# TO DO: Calculate and print test accuracy (1 mark)
multinomial.fit(X_train, y_train)
test_accuracy = multinomial.score(X_test, y_test)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.9555555555555556


### 2.4.2 Tf-idf (6 marks)

To try to improve the results, we can try using Tf-idf to tranform the text data based on the importance of each feature. We will need to use a pipeline and the original data for this section. Use `TfidfVectorizer(min_df=5)` and compare the results for both Logistic Regression and your selected Naive Bayes model. Use the Logistic Regression parameters from the previous section.

In [144]:
# TO DO: Split the data into training and testing sets (same values as previous section) (1 mark)
from sklearn.model_selection import train_test_split

X_train_vect, X_test_vect, y_train_vect, y_test_vect = train_test_split(data.data, data.target, test_size = 0.1, random_state = 0)

In [145]:
# TO DO: Implement Pipeline with Tf-idf vectorizer and both Logistic Regression and your selected Naive Bayes model (3 marks)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

logistic_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 5)),
    ('model', LogisticRegression(max_iter = 1000, C=grid_search.best_params_['C']))
])

multinomial_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 5)),
    ('model', MultinomialNB())
])

logistic_pipeline.fit(X_train_vect, y_train_vect)
multinomial_pipeline.fit(X_train_vect, y_train_vect)

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5)),
                ('model', MultinomialNB())])

In [146]:
# TO DO: Print the results from the grid search (2 marks)
logistic_accuracy = logistic_pipeline.score(X_test_vect, y_test_vect)
multinomial_accuracy = multinomial_pipeline.score(X_test_vect, y_test_vect)
print("Logistic accuracy:", logistic_accuracy)
print("Multinomial accuracy:", multinomial_accuracy)

Logistic accuracy: 0.8666666666666667
Multinomial accuracy: 0.8222222222222222


### Questions (10 marks)

1. Which Naive Bayes model did you pick? Why?
1. Which model and what parameters produced the best results?
1. Was this model a good fit? Why or why not?
1. Is there anything else we could do to try to improve model performance? Provide two ideas (must be different from Part 1).
1. Why did we need to implement a pipeline for Tf-idf and not CountVectorizer? What would happen if we didn't use one for Tf-idf?

1. I chose the Multinomial Naive Bayes model as it is best for text data, which was used here.
2. The best model was MultinomialNB. The validation accuracy was 0.88, and the test and training accuracy were both over 0.95, which is significantly better than the other models tested. 
3. Based on the numerical results, the model fit the training data well and generalized well in cross-validation. It also performed well on the testing set, all of which makes the model a good fit for the data.
4. Two ways to improve the results are to use text-preprocessing to remove unimportant words, or to use multiple models in an ensemble method.
5. A pipeline was implemented for Td-idf because it ensures that the Tf-idf transformation is consistent on both the test and training data. This step is not necessary for CountVectorizer as it does not rely on the datasets global statistics. Without a pipeline, the Tf-idf vectorizer may fit the entire dataset, including the test data, which would change the models performance.

### Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

1. I sourced my code from lecture content, yellowbrick documentation, and generative AI.
2. I completed the steps in the order they were given.
3. I used generative AI to debug and answer questions about models, and any code provided was small enough so as to not require modification.
4. Reading outside documentation and looking through lecture/lab examples was very helpful in understanding the questions being asked in this part.

## Part 3: Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challangeing, motivating
while working on this assignment.


I appreciated the layout and the breakdown of marks in each code cell. I found grid search very intriguing. I also found the model comparison interesting and felt as though it helped elevate my understanding of the various models.